<a href="https://colab.research.google.com/github/buvir/daily_python_practice/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Must-Know API Concepts

A. For ML Engineers (Model Deployment)

Q: How do you expose an ML model as an API?

Example (Flask API for a Salary Predictor):

In [4]:
from sklearn.linear_model import LinearRegression
import pickle

# Sample data
X = [[1], [2], [3]]  # Features
y = [10, 20, 30]     # Labels

# Train and save model
model = LinearRegression()
model.fit(X, y)
pickle.dump(model, open("model.pkl", "wb"))  # Creates 'model.pkl'

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

In [ ]:
model = pickle.load(open("/full/path/to/model.pkl", "rb"))

In [1]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)
model = pickle.load(open("model.pkl", "rb"))  # Pretrained model

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json  # Input: {"experience": 5}
    prediction = model.predict([[data["experience"]]])
    return jsonify({"salary_prediction": prediction[0]})

if __name__ == "__main__":
    app.run()

FileNotFoundError: [Errno 2] No such file or directory: 'model.pkl'

Key Points:

Use POST for sending data to the model.

Return predictions in JSON format.

Tools: Flask, FastAPI, Docker (for containerization).

 For Data Scientists/Analysts (Data Fetching)

Q: How do you fetch data from an API?

Example (Python requests for Weather Data):

In [18]:
import requests
import pandas as pd

url = "https://api.openweathermap.org/data/2.5/weather?q=Chennai&appid=6c6f1989d2637b39c511a75235e4368d"
response = requests.get(url).json()  # Get data as JSON
df = pd.DataFrame([response])  # Convert to DataFrame

In [19]:
df

,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,id,name,cod
0,"{'lon': 80.2785, 'lat': 13.0878}","[{'id': 801, 'main': 'Clouds', 'description': ...",stations,"{'temp': 308.92, 'feels_like': 315.92, 'temp_m...",6000,"{'speed': 4.12, 'deg': 240}",{'all': 20},1747979109,"{'type': 2, 'id': 2094849, 'country': 'IN', 's...",19800,1264527,Chennai,200


In [ ]:
from flask import Flask, request, jsonify
import pickle
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Create and save a model (run this first)
X = [[1], [2], [3]]
y = [10, 20, 30]
model = LinearRegression()
model.fit(X, y)
pickle.dump(model, open("model.pkl", "wb"))

# Step 2: Load the model in Flask
app = Flask(__name__)
model = pickle.load(open("model.pkl", "rb"))

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    prediction = model.predict([[data["experience"]]])
    return jsonify({"prediction": prediction[0]})

if __name__ == "__main__":
    app.run()

In [17]:
import requests
import pandas as pd

# Replace with your actual API key
API_KEY = "6c6f1989d2637b39c511a75235e4368d"  # e.g., "a1b2c3d4e5f6g7h8i9j0"

url = f"https://api.openweathermap.org/data/2.5/weather?q=Chennai&appid={API_KEY}"
response = requests.get(url).json()  # Get data as JSON

if response.get("cod") == 200:  # Check if request succeeded
    df = pd.DataFrame([response])
    print(df[["name", "main", "weather"]])  # Show key fields
else:
    print("Error:", response["message"])  # e.g., "Invalid API key"

      name                                               main  \
0  Chennai  {'temp': 308.92, 'feels_like': 315.92, 'temp_m...   

                                             weather  
0  [{'id': 801, 'main': 'Clouds', 'description': ...  


In [9]:
df

,cod,message
0,401,Invalid API key. Please see https://openweathe...


Proper Data Extraction (Example)

In [ ]:
from pandas import json_normalize

data = {
    "name": response["name"],
    "temp": response["main"]["temp"],
    "humidity": response["main"]["humidity"],
    "weather": response["weather"][0]["description"]
}
df = json_normalize(data)
print(df)

In [16]:
import requests
API_KEY = "6c6f1989d2637b39c511a75235e4368d"  # Replace with actual key
url = f"https://api.openweathermap.org/data/2.5/weather?q=London&appid={API_KEY}"
print(requests.get(url).json())

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 279.22, 'feels_like': 279.22, 'temp_min': 277.9, 'temp_max': 281.13, 'pressure': 1021, 'humidity': 88, 'sea_level': 1021, 'grnd_level': 1017}, 'visibility': 10000, 'wind': {'speed': 0.4, 'deg': 323, 'gust': 0.67}, 'clouds': {'all': 10}, 'dt': 1747978532, 'sys': {'type': 2, 'id': 2091269, 'country': 'GB', 'sunrise': 1747972678, 'sunset': 1748030212}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


In [13]:
import requests
import pandas as pd

def get_weather(api_key, city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()

        if data.get("cod") == 200:
            return pd.DataFrame([{
                "City": data["name"],
                "Temp (°C)": data["main"]["temp"],
                "Humidity": data["main"]["humidity"],
                "Conditions": data["weather"][0]["main"]
            }])
        else:
            print(f"Error {data.get('cod')}: {data.get('message')}")
            print("Solutions:")
            print("- Wait 10 mins if key is new")
            print("- Check key at https://home.openweathermap.org/api_keys")
            return None

    except Exception as e:
        print(f"Request failed: {str(e)}")
        return None

# Usage
API_KEY = "6c6f1989d2637b39c511a75235e4368d"  # Replace with your key
weather_data = get_weather(API_KEY, "Chennai")
if weather_data is not None:
    print(weather_data)

      City  Temp (°C)  Humidity Conditions
0  Chennai      35.74        54     Clouds


In [15]:
mock_data = {
    "name": "Chennai",
    "main": {"temp": 32.5, "humidity": 78},
    "weather": [{"main": "Haze"}]
}